In [16]:
import tensorflow as tf # imortar libreria tensorflow
from tensorflow.examples.tutorials.mnist import input_data # importar en set de datos de mnist

In [17]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True) # a la variable se le pasa el set de datos

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [18]:
print (mnist) #muestra mist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000019977F56630>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000019977F56588>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000019977F56748>)


# Se empieza a construir la grafica computacional

In [19]:
x=tf.placeholder(tf.float32,[None,784]) #imagen de numero decompuesto a un vector de 784 =28x28, el non quiere decir varibale
P=tf.Variable(tf.zeros([784,10])) #matriz de pesos, 784 pixeles para recibir la imagen, 10 posibles salidas
b=tf.Variable(tf.zeros([10])) # vector con bias o sesgo
y=tf.matmul(x,P)+b  #la operacon que se hara en los nodos recibe entradas, matmul es multiplicacion entre matrices
yR=tf.placeholder(tf.float32,[None,10]) # Matriz con las etiquetas reaes del set de datos

In [20]:
#de aqui en adelante se va a entrenar el algoritmo
#definir la  funcion de costo entropia cruzada (cross entropy)para poder medir el error. la salida sera softmax
softmax = tf.nn.softmax_cross_entropy_with_logits(labels=yR,logits=y) #toma el vector de 10 elemntos para hace a prediccion en probailidades
                        #softmax_cross_entropy.. mide el erroro de las predicciones
costo=tf.reduce_mean(softmax) #funcion de costo, buscamos redicir el costo
optimizador=tf.train.GradientDescentOptimizer(0.5).minimize(costo) #le decimos que minimice el costo ajustando los pesos en la matriz

In [21]:
# correr la grafica computacional
prediccion = tf.equal(tf.argmax(y, 1),tf.argmax(yR,1)) #nos da arreglo de booleanos para decirnos cuales estan bien y cuales no
accuracy = tf.reduce_mean(tf.cast(prediccion, tf.float32))  # nos da el procentaje de certeza sobre el arreglo de prediccion
produccion = tf.argmax(y,1)
init = tf.global_variables_initializer()

In [22]:
# Entrenar el algoritmo
# Funcion que usaremos para ver el que tan bien va a aprendiendo nuestro modelo
def avance(epoca_i, sess, last_features, last_labels):
    costoActual = sess.run(costo,feed_dict= {x: last_features, yR:last_labels})
    Certeza = sess.run(accuracy,feed_dict= {x:mnist.validation.images,yR: mnist.validation.labels})
    print('Epoca: {:<4} - Costo: {:<8.3} Certeza: {:<5.3}'.format(epoca_i,costoActual,Certeza))

In [23]:
# esta es la sesion donde realmente se entrena el algoritmo
with tf.Session() as sess:
    sess.run(init)
    for epoca_i in range(1000):
        lotex, lotey = mnist.train.next_batch(100) #se pasan lotes de 100 datos para no saturar
        sess.run(optimizador, feed_dict={x: lotex, yR: lotey})
        if (epoca_i%50==0):
            avance(epoca_i,sess,lotex,lotey)
    print('RESULTADO FINAL: ',sess.run(accuracy, feed_dict={x: mnist.test.images,yR: mnist.test.labels}))
    print('Resutado de una imagen', sess.run(produccion, feed_dict = {x: mnist.test.images[5].reshape(1,784)}))

Epoca: 0    - Costo: 1.71     Certeza: 0.31 
Epoca: 50   - Costo: 0.437    Certeza: 0.882
Epoca: 100  - Costo: 0.447    Certeza: 0.89 
Epoca: 150  - Costo: 0.413    Certeza: 0.901
Epoca: 200  - Costo: 0.237    Certeza: 0.906
Epoca: 250  - Costo: 0.252    Certeza: 0.906
Epoca: 300  - Costo: 0.26     Certeza: 0.911
Epoca: 350  - Costo: 0.318    Certeza: 0.911
Epoca: 400  - Costo: 0.33     Certeza: 0.908
Epoca: 450  - Costo: 0.396    Certeza: 0.914
Epoca: 500  - Costo: 0.445    Certeza: 0.918
Epoca: 550  - Costo: 0.184    Certeza: 0.92 
Epoca: 600  - Costo: 0.377    Certeza: 0.918
Epoca: 650  - Costo: 0.195    Certeza: 0.918
Epoca: 700  - Costo: 0.226    Certeza: 0.919
Epoca: 750  - Costo: 0.204    Certeza: 0.921
Epoca: 800  - Costo: 0.25     Certeza: 0.92 
Epoca: 850  - Costo: 0.339    Certeza: 0.918
Epoca: 900  - Costo: 0.312    Certeza: 0.922
Epoca: 950  - Costo: 0.287    Certeza: 0.921
RESULTADO FINAL:  0.9162
Resutado de una imagen [1]


In [24]:
mnist.test.labels[1]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])